In [1]:
pip install matplotlib


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install pefile==2017.9.3

Note: you may need to restart the kernel to use updated packages.Collecting pefile==2017.9.3
  Using cached pefile-2017.9.3-py3-none-any.whl
  Attempting uninstall: pefile
    Found existing installation: pefile 2019.4.18
    Uninstalling pefile-2019.4.18:
      Successfully uninstalled pefile-2019.4.18




[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install pefile==2019.4.18

  Using cached pefile-2019.4.18-py3-none-any.whl
  Attempting uninstall: pefile
    Found existing installation: pefile 2017.9.3
    Uninstalling pefile-2017.9.3:
      Successfully uninstalled pefile-2017.9.3
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install -U scikit-learn scipy matplotlib

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
# Library
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy
import pickle
import pefile
import sklearn.ensemble as ek
from sklearn.feature_selection import SelectFromModel
import joblib
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn import svm
import sklearn.metrics as metrics
from sklearn.model_selection import cross_val_score
import time
import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [7]:
# Input Dataset 

DM = pd.read_csv("C:\\Users\\Muhammad Raihan\\Downloads\\Obfuscated-MalMem2022-4class.csv") #DM--> Dataset Malware

In [8]:
DM["Category"].value_counts()

Benign        29298
Spyware       10020
Ransomware     9791
Trojan         9487
Name: Category, dtype: int64

In [9]:
#Feature Selection
X = DM.drop(['Category','Class'],axis=1).values    #Droping this because classification model will not accept object type elements (float and int only)
# Target variable
y = DM['Category'].values

In [10]:
#Data Fitting and choosing the important variables
extratrees = ek.ExtraTreesClassifier().fit(X,y)
model = SelectFromModel(extratrees, prefit=True)
X_new = model.transform(X)
nbfeatures = X_new.shape[1]

In [11]:
#Number of important features
nbfeatures

24

In [12]:
features = []
index = numpy.argsort(extratrees.feature_importances_)[::-1][:nbfeatures]

In [13]:
#All the required features
for f in range(nbfeatures):
    print("%d. feature %s (%f)" % (f + 1, DM.columns[2+index[f]], extratrees.feature_importances_[index[f]]))
    features.append(DM.columns[2+f])

1. feature handles.nhandles (0.058070)
2. feature pslist.nprocs64bit (0.057893)
3. feature handles.ndesktop (0.056499)
4. feature dlllist.avg_dlls_per_proc (0.054720)
5. feature ldrmodules.not_in_init (0.051525)
6. feature ldrmodules.not_in_load (0.044972)
7. feature handles.ndirectory (0.040168)
8. feature ldrmodules.not_in_load_avg (0.037436)
9. feature ldrmodules.not_in_mem (0.033328)
10. feature svcscan.interactive_process_services (0.033277)
11. feature handles.nthread (0.031765)
12. feature handles.ntimer (0.031251)
13. feature handles.nsection (0.030827)
14. feature dlllist.ndlls (0.030564)
15. feature handles.nport (0.029078)
16. feature handles.avg_handles_per_proc (0.027344)
17. feature malfind.ninjections (0.026908)
18. feature ldrmodules.not_in_init_avg (0.026164)
19. feature svcscan.shared_process_services (0.024939)
20. feature handles.nmutant (0.024387)
21. feature ldrmodules.not_in_mem_avg (0.022665)
22. feature handles.nevent (0.021175)
23. feature pslist.avg_threads (

In [14]:
class NeuralDecisionTree(keras.Model):
    def __init__(self, depth, num_features, used_features_rate, num_classes):
        super().__init__()
        self.depth = depth
        self.num_leaves = 2 ** depth
        self.num_classes = num_classes

        # Create a mask for the randomly selected features.
        num_used_features = int(num_features * used_features_rate)
        one_hot = np.eye(num_features)
        sampled_feature_indicies = np.random.choice(
            np.arange(num_features), num_used_features, replace=False
        )
        self.used_features_mask = one_hot[sampled_feature_indicies]

        # Initialize the weights of the classes in leaves.
        self.pi = tf.Variable(
            initial_value=tf.random_normal_initializer()(
                shape=[self.num_leaves, self.num_classes]
            ),
            dtype="float32",
            trainable=True,
        )

        # Initialize the stochastic routing layer.
        self.decision_fn = layers.Dense(
            units=self.num_leaves, activation="sigmoid", name="decision"
        )

    def call(self, features):
        batch_size = tf.shape(features)[0]

        # Apply the feature mask to the input features.
        features = tf.matmul(
            features, self.used_features_mask, transpose_b=True
        )  # [batch_size, num_used_features]
        # Compute the routing probabilities.
        decisions = tf.expand_dims(
            self.decision_fn(features), axis=2
        )  # [batch_size, num_leaves, 1]
        # Concatenate the routing probabilities with their complements.
        decisions = layers.concatenate(
            [decisions, 1 - decisions], axis=2
        )  # [batch_size, num_leaves, 2]

        mu = tf.ones([batch_size, 1, 1])

        begin_idx = 1
        end_idx = 2
        # Traverse the tree in breadth-first order.
        for level in range(self.depth):
            mu = tf.reshape(mu, [batch_size, -1, 1])  # [batch_size, 2 ** level, 1]
            mu = tf.tile(mu, (1, 1, 2))  # [batch_size, 2 ** level, 2]
            level_decisions = decisions[
                :, begin_idx:end_idx, :
            ]  # [batch_size, 2 ** level, 2]
            mu = mu * level_decisions  # [batch_size, 2**level, 2]
            begin_idx = end_idx
            end_idx = begin_idx + 2 ** (level + 1)

        mu = tf.reshape(mu, [batch_size, self.num_leaves])  # [batch_size, num_leaves]
        probabilities = keras.activations.softmax(self.pi)  # [num_leaves, num_classes]
        outputs = tf.matmul(mu, probabilities)  # [batch_size, num_classes]
        return outputs

In [15]:
class NeuralDecisionForest(keras.Model):
    def __init__(self, num_trees, depth, num_features, used_features_rate, num_classes):
        super().__init__()
        self.ensemble = []
        # Initialize the ensemble by adding NeuralDecisionForest instances.
        # Each tree will have its own randomly selected input features to use.
        for _ in range(num_trees):
            self.ensemble.append(
                NeuralDecisionTree(depth, num_features, used_features_rate, num_classes)
            )

    def call(self, inputs):
        # Initialize the outputs: a [batch_size, num_classes] matrix of zeros.
        batch_size = tf.shape(inputs)[0]
        outputs = tf.zeros([batch_size, num_classes])

        # Aggregate the outputs of trees in the ensemble.
        for tree in self.ensemble:
            outputs += tree(inputs)
        # Divide the outputs by the ensemble size to get the average.
        outputs /= len(self.ensemble)
        return outputs

In [20]:
# Assuming 'DM' is your DataFrame
X = DM[features]
y = DM['Category'].astype('category').cat.codes

# Convert y to a 1D array and make sure it's an integer array
y = y.values.astype(int)

# Define the number of folds
num_folds = 10
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)
learning_rate = 0.01
num_classes = 4  
batch_size = 265
num_epochs = 100
num_trees = 25
depth = 5
used_features_rate = 0.5

# Function to create the forest model
def create_forest_model():
    input_shape = X.shape[1]
    inputs = layers.Input(shape=(input_shape,))
    normalized_inputs = layers.BatchNormalization()(inputs)

    forest_model = NeuralDecisionForest(
        num_trees, depth, input_shape, used_features_rate, num_classes=num_classes
    )

    outputs = forest_model(normalized_inputs)
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

# Function to run the experiment with cross-validation
def run_experiment_with_cv(model, X, y, skf):
    fold_results = []

    for fold, (train_index, test_index) in enumerate(skf.split(X, y), 1):
        print(f"\nFold {fold}:")

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y[train_index], y[test_index]

        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss=keras.losses.SparseCategoricalCrossentropy(),
            metrics=[keras.metrics.SparseCategoricalAccuracy()],
        )

        print("Start training the model...")
        start_time = time.time()
        model.fit(X_train, y_train, epochs=num_epochs, batch_size=batch_size, validation_split=0.2)
        end_time = time.time()
        print("Model training finished")

        print("Evaluating the model on the test data...")
        _, accuracy = model.evaluate(X_test, y_test)
        print(f"Test accuracy: {round(accuracy * 100, 2)}%")

        y_test = y_test.astype(int)
        y_pred = np.argmax(model.predict(X_test), axis=1)

        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, average='weighted')
        recall = recall_score(y_test, y_pred, average='weighted')
        f1 = f1_score(y_test, y_pred, average='weighted')

        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")

        training_time = end_time - start_time
        print("Total Training Time:", training_time, "seconds")

        fold_results.append({
            "fold": fold,
            "accuracy": accuracy,
            "precision": precision,
            "recall": recall,
            "f1": f1,
            "training_time": training_time,
        })

    return fold_results

# Run the experiment with cross-validation
results = run_experiment_with_cv(create_forest_model(), X, y, skf)

# Display average performance metrics across all folds
avg_accuracy = np.mean([result["accuracy"] for result in results])
avg_precision = np.mean([result["precision"] for result in results])
avg_recall = np.mean([result["recall"] for result in results])
avg_f1 = np.mean([result["f1"] for result in results])
avg_training_time = np.mean([result["training_time"] for result in results])

print("\nAverage Performance Metrics Across Folds:")
print(f"Average Accuracy: {avg_accuracy:.4f}")
print(f"Average Precision: {avg_precision:.4f}")
print(f"Average Recall: {avg_recall:.4f}")
print(f"Average F1 Score: {avg_f1:.4f}")
print(f"Average Training Time: {avg_training_time:.4f} seconds")


Fold 1:
Start training the model...
Epoch 1/100
160/160 [==============================] - 13s 17ms/step - loss: 0.7342 - sparse_categorical_accuracy: 0.8192 - val_loss: 2.7696 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 2/100
160/160 [==============================] - 1s 7ms/step - loss: 0.4110 - sparse_categorical_accuracy: 0.8299 - val_loss: 2.1253 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 3/100
160/160 [==============================] - 1s 7ms/step - loss: 0.3736 - sparse_categorical_accuracy: 0.8331 - val_loss: 2.0624 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 4/100
160/160 [==============================] - 1s 8ms/step - loss: 0.3578 - sparse_categorical_accuracy: 0.8343 - val_loss: 2.0324 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 5/100
160/160 [==============================] - 1s 8ms/step - loss: 0.3484 - sparse_categorical_accuracy: 0.8349 - val_loss: 2.0621 - val_sparse_categorical_accuracy: 0.0000e+00
Epoch 6/100
160/160 [===============